In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
x_test = pd.read_csv('titanic/test.csv')
x_train  = pd.read_csv('titanic/train.csv')
x_test_copy = pd.read_csv('titanic/test copy.csv')
x_test.head(5)
# x_train.sample(5)
x_test.describe()
x_train.sample(5)
# x_train.info()

In [ ]:
x_train.drop(['PassengerId', 'Ticket', 'Cabin','Name'],'columns' ,inplace=True)
x_train.info()


In [ ]:
x_train[x_train.isnull().any(axis=1)].count()

In [ ]:
x_train= x_train.dropna()

In [ ]:
pd.crosstab(x_train['Sex'], x_train['Survived'])

In [ ]:
pd.crosstab(x_train['Pclass'], x_train['Survived'])

In [ ]:
# pd.crosstab(x_train['Age'], x_train['Survived'])

In [ ]:
x_train_corr = x_train.corr()
x_train_corr

* To look at correlation data draw a heatmap

In [ ]:
import seaborn as sns
fig, ax = plt.subplots(figsize=(12,10))
sns.heatmap(x_train_corr, annot=True)

In [ ]:
from sklearn import preprocessing
label_encoding = preprocessing.LabelEncoder()
x_train['Sex'] = label_encoding.fit_transform(x_train['Sex'].astype(str))
label_encoding.classes_

In [ ]:
x_train = pd.get_dummies(x_train, columns=['Embarked'])

In [ ]:
x_train.drop(columns='Embark', axis=1, inplace=True)

In [ ]:
x_train.head(5)

In [ ]:
x_train = x_train.sample(frac=1).reset_index(drop=True)
x_train.head()

In [ ]:
x_train.to_csv('titanic/titanic_processed.csv', index=False)

In [ ]:
!ls titanic

In [ ]:
titanic_dt = pd.read_csv('titanic/titanic_processed.csv')
titanic_dt.head()

In [ ]:
from sklearn.model_selection import train_test_split
X = titanic_dt.drop('Survived',axis=1)
Y = titanic_dt['Survived']

x_train, x_test, y_train, y_test  = train_test_split(X, Y, test_size=0.2)

In [ ]:
from sklearn.linear_model import LogisticRegression
logistic_model = LogisticRegression(penalty='l2',C=2.0, solver='liblinear').fit(x_train, y_train)


In [ ]:
y_pred = logistic_model.predict(x_test)

In [ ]:
pred_results = pd.DataFrame({'y_test':y_test, 'y_pred':y_pred})

In [ ]:
pred_results.sample(5)

In [ ]:
titanic_crosstab = pd.crosstab(pred_results.y_pred, pred_results.y_test)

In [ ]:
titanic_crosstab

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [ ]:
acc= accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

In [ ]:
print(f"The Accuracy :: {acc}")
print(f"Precision :: {prec}")
print(f"The recall :: {recall}")

# `Multiple Classification Models`

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier

In [ ]:
titanic_df = pd.read_csv('titanic/titanic_processed.csv')

In [ ]:
FEATURES = list(titanic_df.columns[1:])
FEATURES

In [ ]:
result_dict = {}


In [ ]:
def summarize_classification(y_test, y_pred):
    acc = accuracy_score(y_test, y_pred, normalize=True)
    num_acc = accuracy_score(y_test, y_pred, normalize=False)# number of accurately predicted scores
    prec= precision_score(y_test, y_pred)
    recall = precision_score(y_test, y_pred)

    return {'accuracy':acc,
            'precision':prec,
            'recall':recall,
            'accuracy_count':num_acc
    }

In [ ]:
def build_model(classifier_fn, dataset, column_name, column_names, test_size=0.2,):
    X = dataset[column_names]
    Y = dataset[column_name]

    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = test_size)

    model = classifier_fn(x_train, y_train)

    y_pred = model.predict(x_test)

    y_pred_train = model.predict(x_train)

    test_summary = summarize_classification(y_test, y_pred)
    train_summary = summarize_classification(y_train, y_pred_train)

    pred_results = pd.DataFrame({'y_test':y_test,
    'y_pred':y_pred})

    model_crosstab = pd.crosstab(pred_results.y_pred, pred_results.y_test)

    return {'training': train_summary,
            'test': test_summary,
            'confusion_matrix':model_crosstab
    }


    



In [ ]:
def compare_results():
    for key in result_dict:
        print("Classification: ", key)

        print()
        print("Training data")
        for score in result_dict[key]['training']:
            print(score, result_dict[key]['training'][score])

        print()
        print('Test data')
        for score in result_dict[key]['test']:
            print(score, result_dict[key]['test'][score])

        print()

In [ ]:
def logistic_fn(x_train,y_train):
    model = LogisticRegression(solver='liblinear', C=2.0, penalty='l2' )
    model.fit(x_train, y_train)
    return model

In [ ]:
result_dict['survived ~ logistic'] = build_model(logistic_fn, titanic_df, 'Survived', FEATURES, )
compare_results()

In [ ]:
def linear_discriminant_fn(x_train, y_train, solver="svd"):
    
    model = LinearDiscriminantAnalysis(solver=solver)
    model.fit(x_train, y_train)

    return model

In [ ]:
result_dict['survived ~ linear_discriminant_analysis'] = build_model(linear_discriminant_fn,titanic_df,
"Survived", FEATURES[0:-1])

compare_results()

In [ ]:
def quadratic_discriminant_fn(x_train, y_train):
    model = QuadraticDiscriminantAnalysis()
    model.fit(x_train, y_train)

    return model

In [ ]:
result_dict['survived ~ quadratic_discriminant_analysis'] = build_model(quadratic_discriminant_fn, titanic_df, 'Survived', FEATURES[0:-1])
compare_results()

In [ ]:
def sgd_fn(x_train, y_train, max_iter = 100000, tol=1e-3):

    model = SGDClassifier(max_iter=max_iter, tol=tol)
    model.fit(x_train, y_train)

    return model

In [ ]:
result_dict['survived ~ sgd'] = build_model(sgd_fn, titanic_df, 'Survived', FEATURES)
compare_results()

In [ ]:
def linear_svc_fn(x_train, y_train, C=.3, max_iter = 70000, tol = 1e-3):

    model = LinearSVC(C=C, tol=tol, max_iter=max_iter, dual=False)
    model.fit(x_train, y_train)
    return model

In [ ]:
result_dict['Classification ~ svd'] = build_model(linear_svc_fn, titanic_df, 'Survived', FEATURES)
compare_results()


In [ ]:
def radius_neighbor_fn(x_train, y_train, radius=40.0):
    model = RadiusNeighborsClassifier(radius=radius)
    model.fit(x_train, y_train)

    return model

In [ ]:
result_dict['Classification ~ RadiusNeighborsClassifier'] = build_model(radius_neighbor_fn, titanic_df, 'Survived', FEATURES)
compare_results()

In [ ]:
def decision_tree_fn(x_train, y_train, max_depth=None, max_features = None):
    model = DecisionTreeClassifier(max_depth=max_depth,max_features=max_features)
    model.fit(x_train, y_train)
    return model

In [ ]:
result_dict['Classification ~ decision_tree'] = build_model(decision_tree_fn, titanic_df, 'Survived', FEATURES)
compare_results()

In [ ]:
def naive_bayes_fn(x_train, y_train, priors=None):

    model = GaussianNB(priors=priors)
    model.fit(x_train, y_train)

    return model

In [ ]:
result_dict['Classification ~ naive_bayes'] = build_model(naive_bayes_fn, titanic_df, "Survived", FEATURES)
compare_results()